In [1]:
from pyspark import SparkContext
from geopyspark.geopycontext import GeoPyContext
from geopyspark.geotrellis.catalog import read, read_value, query, write
from geopyspark.geotrellis.constants import SPATIAL, ZOOM, TILE
from geopyspark.geotrellis.geotiff_rdd import get
from geopyspark.geotrellis.rdd import RasterRDD, TiledRasterRDD
from geonotebook.vis.geotrellis.render_methods import render_nlcd, single_band_render_from_color_map
from geonotebook.wrappers import GeoTrellisCatalogLayerData, RddRasterData
import numpy as np

## Vieweing NLCD

In [2]:
M.set_center(-120.32, 47.84, 7)

In [4]:
sc = SparkContext(appName="NLCD Viewer")
geopysc = GeoPyContext(sc)

In [5]:
catalog_uri = "s3://azavea-datahub/catalog"
layer_name = "us-ned-tms-epsg3857"

In [6]:
data = GeoTrellisCatalogLayerData(geopysc, 
                                  catalog_uri, 
                                  layer_name,
                                  SPATIAL)

In [7]:
from geonotebook.vis.geotrellis.render_methods import render_nlcd
from geopyspark.geotrellis.render import ColorRamp

cmap1 = ColorRamp.getHex(geopysc, "viridis", 50)
colors = dict(enumerate(cmap1, start=1))
    
cmap_render1 = single_band_render_from_color_map(colors)
def render_ed(tile):
    return cmap_render1(tile[0].astype(int))

M.add_layer(data,render_tile=render_ed)

::1 - - [2017-05-19 19:51:26] "GET /tile/4/7/7.png HTTP/1.1" 500 2940 1.246742
::1 - - [2017-05-19 19:51:26] "GET /tile/4/7/8.png HTTP/1.1" 500 2940 0.036779
::1 - - [2017-05-19 19:51:26] "GET /tile/4/8/7.png HTTP/1.1" 500 2940 0.026858
::1 - - [2017-05-19 19:51:26] "GET /tile/4/6/8.png HTTP/1.1" 500 2940 0.121229
::1 - - [2017-05-19 19:51:26] "GET /tile/4/8/6.png HTTP/1.1" 500 2940 0.025681
::1 - - [2017-05-19 19:51:27] "GET /tile/4/7/6.png HTTP/1.1" 500 2940 0.074868
::1 - - [2017-05-19 19:51:27] "GET /tile/4/8/9.png HTTP/1.1" 500 2940 0.024157
::1 - - [2017-05-19 19:51:27] "GET /tile/4/9/7.png HTTP/1.1" 500 2940 0.025273
::1 - - [2017-05-19 19:51:27] "GET /tile/4/9/8.png HTTP/1.1" 500 2940 0.036467
::1 - - [2017-05-19 19:51:27] "GET /tile/4/9/6.png HTTP/1.1" 500 2940 0.088020
::1 - - [2017-05-19 19:51:27] "GET /tile/4/6/6.png HTTP/1.1" 500 2940 0.076155
::1 - - [2017-05-19 19:51:27] "GET /tile/4/6/9.png HTTP/1.1" 500 2940 0.026458
::1 - - [2017-05-19 19:51:27] "GET /tile/4/9/9.png H

::1 - - [2017-05-19 19:51:44] "GET /tile/4/1/1.png HTTP/1.1" 500 2778 0.183079
::1 - - [2017-05-19 19:51:44] "GET /tile/4/15/3.png HTTP/1.1" 500 2780 0.067852
::1 - - [2017-05-19 19:51:44] "GET /tile/4/2/1.png HTTP/1.1" 500 2778 0.067180
::1 - - [2017-05-19 19:51:44] "GET /tile/4/15/4.png HTTP/1.1" 500 2780 0.069032
::1 - - [2017-05-19 19:51:44] "GET /tile/4/0/1.png HTTP/1.1" 500 2778 0.074133
::1 - - [2017-05-19 19:51:44] "GET /tile/4/15/2.png HTTP/1.1" 500 2780 0.074113
::1 - - [2017-05-19 19:51:44] "GET /tile/4/15/1.png HTTP/1.1" 500 2780 0.070905
::1 - - [2017-05-19 19:51:44] "GET /tile/4/15/5.png HTTP/1.1" 500 2780 0.071880
::1 - - [2017-05-19 19:51:44] "GET /tile/4/3/1.png HTTP/1.1" 500 2778 0.072087
::1 - - [2017-05-19 19:51:45] "GET /tile/5/2/7.png HTTP/1.1" 500 2778 0.197836
::1 - - [2017-05-19 19:51:45] "GET /tile/5/1/7.png HTTP/1.1" 200 3140 0.262687
::1 - - [2017-05-19 19:51:45] "GET /tile/5/1/6.png HTTP/1.1" 200 1392 0.260592
::1 - - [2017-05-19 19:51:45] "GET /tile/5/2/6.

## Viewing reclassified tiles

In [ ]:
def reclass(tile):
    # Planted/Cultivated
    # See https://www.mrlc.gov/nlcd11_leg.php
    result = tile.copy()
    result[np.ma.where((80 <= tile) & (tile < 90))] = 1
    result[np.ma.where((tile  < 80) | (90 <= tile))] = 0
    return result
     
cmap = { 0 : "#00000000", 1: "#CA9146FF" }
cmap_render = single_band_render_from_color_map(cmap)

def reclass_render(tile):
    reclassed = reclass(tile[0])
    return cmap_render(reclassed)

In [24]:
M.layers

<GeonotebookLayerCollection([<InProcessTileLayer('-2302025634942909807__us-ned-tms-epsg3857')>])>

In [39]:
M.add_layer(data, render_tile=reclass_render)

NameError: name 'reclass_render' is not defined

## Chattanooga geometry

In [ ]:
!curl -o /tmp/mask.json https://s3.amazonaws.com/chattademo/chatta_mask.json

In [ ]:
from functools import partial
import fiona
import json
import pyproj
from shapely.geometry import mapping, shape
from shapely.ops import transform

project = partial(
    pyproj.transform,
    pyproj.Proj(init='epsg:4326'),
    pyproj.Proj(init='epsg:3857'))

txt = open('/tmp/mask.json').read()
js = json.loads(txt)
geom = shape(js)
center = geom.centroid
chatta_poly = transform(project, geom)
chatta_poly

In [40]:
M.remove_layer(M.layers[0])

In [ ]:
from geonotebook.wrappers import VectorData
vd = VectorData("/tmp/mask.json")
name = "Outline"
M.add_layer(vd, name=name)

In [ ]:
M.set_center(center.x, center.y, 9)

## Fetching an RDD of NLCD masked to Chattanooga

In [ ]:
MAX_ZOOM = 12
query_rdd = query(geopysc, SPATIAL, catalog_uri, layer_name, 12, intersects=chatta_poly)
converted_rdd = query_rdd.convert_data_type("int8")

In [ ]:
masked = converted_rdd.mask([chatta_poly])
rd = RddRasterData(masked)

In [29]:
M.remove_layer(M.layers[0])

In [ ]:
M.add_layer(rd, render_tile=render_nlcd)

## Reclassifying an RDD

In [ ]:
nprdd = converted_rdd.to_numpy_rdd()

In [ ]:
def mapper(tile):
    arr = tile['data'][0]
    np.ma.masked_where(arr == tile['no_data_value'], arr)
    tile['data'] = np.array([reclass(arr)])
    return tile
mapped = nprdd.mapValues(mapper).cache()
gtRdd = TiledRasterRDD.from_numpy_rdd(geopysc, SPATIAL, mapped, query_rdd.layer_metadata)
reclassed = gtRdd.mask([chatta_poly])

In [32]:
cmap = { 0 : "#00FFAA88", 100000: "#CA9146FF" }
cmap_render = single_band_render_from_color_map(cmap)
def render_tile(tile):
    arr = tile[0]
    print(arr.min(), arr.max())
    arr[:] = 100000
    return cmap_render(arr)

In [ ]:
rd = RddRasterData(reclassed)

In [34]:
M.remove_layer(M.layers[0])

In [ ]:
M.add_layer(rd, render_tile=render_tile)

## Saving the reclassified layer locally

In [ ]:
local_catalog_uri = "file://catalog"

In [ ]:
# Reproject to bring the tile sizes to 256
retiled = reclassed.reproject("EPSG:3857", scheme=ZOOM)

In [ ]:
for layer_rdd in retiled.pyramid(retiled.zoom_level, 0):
    write(local_catalog_uri, "cultivated-land-cover", layer_rdd)


## Viewing the local layer

In [ ]:
data = GeoTrellisCatalogLayerData(geopysc, 
                                  local_catalog_uri, 
                                  "cultivated-land-cover",
                                  SPATIAL)

In [ ]:
M.remove_layer(M.layers[0])

In [ ]:
M.add_layer(data, render_tile=render_tile)